## Prepare data reader and model execution context

In [1]:

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.blockbatch.construct import construct_data_readers
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "refined_traces"

# Define the input data range.
readers: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS,
)


# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0] # use the first reader
)


2025-01-07 11:42:57 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=84882
2025-01-07 11:42:57 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=84882
2025-01-07 11:42:57 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76 process=84882
2025-01-07 11:42:57 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=84882
2025-01-07 11:42:57 [info     ] prepared 22 input batches.     filename=byblock.py lineno=85 process=84882
2025-01-07 11:42:57 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/c36_jia6/op-analytics.duck.db filename=client.py lineno=62 process=84882
2025-01-07 11:42:57 [info     ] reading dataset='ingestion/blocks_v1' using 1/1 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=84882
2025-01-07 11:42:57 [info     ] r

## Execute the model

It's up to you how the model manipulates the data. 

Develop the various steps in notebook cells and then copy the final code over to the model function.

In [2]:
refined_txs_table = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_traces_projection = aux_views["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_amortized = aux_views["refined_traces/traces_amortized"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)

traces_txs_join = aux_views["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_amortized": traces_amortized,
        "refined_transactions_fees": refined_txs_table,
    },
)

2025-01-07 11:42:57 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=222 process=84882
2025-01-07 11:42:57 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=222 process=84882
2025-01-07 11:42:57 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=84882 template=refined_transactions_fees
2025-01-07 11:43:02 [info     ] duck db size: 8.4MB            filename=client.py lineno=34 process=84882
2025-01-07 11:43:02 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=222 process=84882
2025-01-07 11:43:02 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=84882 template=refined_traces/traces_projection
2025-01-07 11:43:07 [info     ] duck db size: 59.0MB           filename=client.py lineno=34 process=84882


TemplateNotFound: refined_traces/traces_amortized.sql.j2

In [3]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x455d3fa505b6ff90bdb6385735c258a399d21c9d46c0a457c9a10505e09e925b'
ORDER BY trace_address
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬──────────────┬─────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────────

In [5]:
ctx.client.sql(f"""DESCRIBE {refined_txs_table}""").show(max_rows=100)

┌─────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │  column_type   │  null   │   key   │ default │  extra  │
│               varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                  │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                               │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                            │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                             │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                               │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index                   │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                